In [5]:
# rm(list=ls())
require(feather, lib.loc = "/homes/sadatnfs/R/x86_64-unknown-linux-gnu-library/3.1")
require(data.table)
require(reshape2)
require(MASS)
require(ggplot2)
require(foreach)

Loading required package: feather
Loading required package: data.table
Loading required package: reshape2

Attaching package: ‘reshape2’

The following objects are masked from ‘package:data.table’:

    dcast, melt

Loading required package: MASS
Loading required package: ggplot2


In [6]:
pop <- fread("/home/j/Project/IRH/Forecasting/data/WPP_forecasted_population.csv")
pop <- data.table(pop)

In [91]:
## Education forecasts
# edu_orig<-data.table(fread(paste0("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws.csv"), header = T))
edu<-data.table(fread(paste0("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/intermediate/gpr_draws_wide_by_years.csv"), header = T))
# edu <- data.table(edu)

Read 496860 rows and 1005 (of 1005) columns from 6.600 GB file in 00:02:11


### Reshape long by draws, wide by year, use copula with spearman = 1, and reshape back AND collapse by country-year

### Neal's Copula Function (rho = 1)

In [8]:
rank_time_series <- function(X, corr, ts=F, print=FALSE){
  draws <- nrow(X)
  years <- ncol(X)
  # X <- matrix(runif(draws*years), nrow=years, ncol=draws)
  
  if(print){
    print(sapply(2:years, function(x) cor(X[x,], X[x-1,])))
  }
  
  if(ts == F) {
    corr_mat <- matrix(data=corr, nrow= years, ncol= years )
    diag(corr_mat) <- 1
  }
  
  if(ts == T) {
    corr_mat <- corr**abs(outer(0:(years-1), 0:(years-1), "-"))
  }
  
  
#   print(corr_mat)
  mvdat <- mvrnorm(n= draws, mu=0 * 1:ncol(X), Sigma=corr_mat, empirical=T)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sorted_X <- apply(X, 2, sort)
  sapply(1:ncol(X), function(x) sorted_X[,x][ranks[,x]])
}



### Let's differentiate each chunks of unique loc_id, age, sex

In [13]:
# locs <- unique(edu[,location_id])
# ages <- unique(edu[,age_group_id])
# sexs <- unique(edu[,sex_id])

edu <- edu[, id := .GRP, by = .(location_id, age_group_id, sex_id)]
edu <- setcolorder(edu, c("id", colnames(edu)[1:dim(edu)[2]-1]))

## Unique ID var
unique<- unique(edu[,id])
head(edu)

### Use copula on all the data with Spearman = 1

In [73]:
Start<- Sys.time()
tempx <-data.table(foreach(i = unique , .combine = 'rbind') %dopar% rank_time_series(edu[id==i, 6:96], corr = 1, ts = F,print=F))
colnames(tempx) <- paste0(c(1950:2040))
# head(tempx)
print(Sys.time()-Start)

1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,⋯,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
3.257842,3.390548,3.520781,3.643780,3.788279,3.909383,4.059078,4.240794,4.391472,4.559399,⋯,11.883740,11.97155,12.038046,12.106941,12.163648,12.223369,12.269799,12.347869,12.417393,12.504811
5.437501,5.562104,5.708561,5.851417,5.998803,6.115347,6.241912,6.358497,6.513877,6.662712,⋯,13.715482,13.81627,13.954534,14.032046,14.111295,14.200654,14.295447,14.382608,14.467780,14.564952
3.151690,3.263616,3.402498,3.545805,3.692283,3.835464,3.967979,4.101882,4.254849,4.422516,⋯,11.770790,11.83054,11.911594,11.974531,12.036385,12.105070,12.171925,12.227402,12.283675,12.341148
7.666157,7.750208,7.832157,7.959294,8.051388,8.187833,8.334424,8.465775,8.732138,8.762940,⋯,15.030493,15.14849,15.288745,15.359190,15.461716,15.550453,15.635890,15.718701,15.795828,15.852173
1.863376,1.928312,2.022635,2.103857,2.203690,2.316012,2.429992,2.539441,2.658950,2.800168,⋯,9.532726,9.47913,9.457744,9.424406,9.472533,9.549037,9.535574,9.524559,9.574898,9.614353
3.122339,3.240621,3.387369,3.518082,3.639210,3.797781,3.950234,4.079835,4.226543,4.380017,⋯,11.726654,11.80076,11.876166,11.936391,12.023070,12.069855,12.137498,12.186018,12.236115,12.299342


Time difference of 28.06799 mins


In [74]:
Sys.time()
el_bindo <- cbind(edu[, .(id, location_id, age_group_id, sex_id, variable)], tempx)
el_bindo <- el_bindo[, id:=NULL]
Sys.time()

[1] "2017-01-28 14:58:43 PST"

[1] "2017-01-28 14:58:47 PST"

### Save out to reshape wide in Pandas

In [ ]:
fwrite(el_bindo, "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/intermediate/gpr_draws_wide_by_years_sorted.csv")

### Try in here first?

In [77]:
blank2_melt <- melt(el_bindo, id.vars = c("location_id", "age_group_id", "sex_id", "variable"), variable.name = "year_id", value.name = "data")

In [79]:
Start <- Sys.time()
blank2_cast <- dcast(blank2_melt, location_id + age_group_id + sex_id + year_id ~ variable )
print(Sys.time() - Start)

Using data as value column: use value.var to override.


Time difference of 10.24608 mins


In [81]:
head(blank2_cast)

location_id,age_group_id,sex_id,year_id,draw0,draw1,draw10,draw100,draw101,draw102,⋯,draw990,draw991,draw992,draw993,draw994,draw995,draw996,draw997,draw998,draw999
6,8,1,1950,3.257842,5.437501,2.513109,3.685016,4.408052,4.262174,⋯,6.773038,4.947084,2.547622,5.358931,3.534418,1.884700,3.853974,7.299674,4.724487,11.60851
6,8,1,1951,3.390548,5.562104,2.647629,3.824688,4.521345,4.416285,⋯,6.886386,5.080068,2.680469,5.467550,3.679010,1.991216,3.969894,7.417908,4.805875,11.64631
6,8,1,1952,3.520781,5.708561,2.749598,3.934128,4.676883,4.534475,⋯,7.017713,5.231352,2.800236,5.621149,3.812673,2.083651,4.107590,7.580773,4.929192,11.66306
6,8,1,1953,3.643780,5.851417,2.860612,4.063825,4.805242,4.656573,⋯,7.133385,5.385191,2.928341,5.763801,3.973355,2.182793,4.239279,7.737209,5.080461,11.65999
6,8,1,1954,3.788279,5.998803,2.973653,4.230364,4.925693,4.784819,⋯,7.257398,5.496507,3.014197,5.910079,4.131252,2.267882,4.406038,7.874478,5.233688,11.63845
6,8,1,1955,3.909383,6.115347,3.080803,4.406265,5.059170,4.901445,⋯,7.415128,5.610584,3.134932,6.046451,4.268343,2.367576,4.544663,8.025699,5.368539,11.59986


In [82]:
fwrite(blank2_cast, "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws_country_year_age_sex_post_copula.csv")

### Collapse to year and country only

In [83]:
edu2 <- merge(blank2_cast, pop, by = c("location_id", "year_id", "age_group_id", "sex_id"))

In [84]:
edu4<-data.table(edu2)[, lapply(.SD, function(x) sum(x*pop)/sum(pop)), 
    by=c("location_id", "year_id"), .SDcols=cbind(paste0("draw",c(0:999))) ] 
head(edu4)

location_id,year_id,draw0,draw1,draw2,draw3,draw4,draw5,draw6,draw7,⋯,draw990,draw991,draw992,draw993,draw994,draw995,draw996,draw997,draw998,draw999
10,1950,1.747954,1.778849,1.797826,1.676938,1.553651,1.856006,1.388680,1.402455,⋯,2.078309,1.409418,1.802613,1.687683,1.897167,1.766918,1.772139,1.774524,1.522151,1.393033
10,1951,1.773521,1.808386,1.828858,1.702910,1.587144,1.884090,1.418387,1.426755,⋯,2.108773,1.442241,1.827847,1.717511,1.934726,1.802392,1.804692,1.802341,1.552032,1.423956
10,1952,1.804439,1.838140,1.862759,1.732470,1.624615,1.909133,1.447499,1.453348,⋯,2.157524,1.473800,1.861630,1.747652,1.967994,1.834619,1.837122,1.833001,1.580961,1.457321
10,1953,1.834183,1.868393,1.891366,1.764041,1.654839,1.936369,1.475651,1.480071,⋯,2.192936,1.508563,1.894210,1.776561,2.004377,1.863849,1.872104,1.866471,1.613674,1.492473
10,1954,1.867477,1.899370,1.923786,1.803405,1.687248,1.981079,1.507094,1.512784,⋯,2.210855,1.544202,1.931476,1.809469,2.043258,1.898874,1.910405,1.903951,1.651653,1.530006
10,1955,1.903475,1.933966,1.961608,1.832743,1.725020,2.010999,1.537940,1.545754,⋯,2.250129,1.578653,1.962877,1.845211,2.080186,1.937068,1.946695,1.937738,1.687500,1.565150


### Copula this one?

In [180]:
head(edu4)

location_id,year_id,draw0,draw1,draw2,draw3,draw4,draw5,draw6,draw7,⋯,draw990,draw991,draw992,draw993,draw994,draw995,draw996,draw997,draw998,draw999
10,1950,1.747954,1.778849,1.797826,1.676938,1.553651,1.856006,1.388680,1.402455,⋯,2.078309,1.409418,1.802613,1.687683,1.897167,1.766918,1.772139,1.774524,1.522151,1.393033
10,1951,1.773521,1.808386,1.828858,1.702910,1.587144,1.884090,1.418387,1.426755,⋯,2.108773,1.442241,1.827847,1.717511,1.934726,1.802392,1.804692,1.802341,1.552032,1.423956
10,1952,1.804439,1.838140,1.862759,1.732470,1.624615,1.909133,1.447499,1.453348,⋯,2.157524,1.473800,1.861630,1.747652,1.967994,1.834619,1.837122,1.833001,1.580961,1.457321
10,1953,1.834183,1.868393,1.891366,1.764041,1.654839,1.936369,1.475651,1.480071,⋯,2.192936,1.508563,1.894210,1.776561,2.004377,1.863849,1.872104,1.866471,1.613674,1.492473
10,1954,1.867477,1.899370,1.923786,1.803405,1.687248,1.981079,1.507094,1.512784,⋯,2.210855,1.544202,1.931476,1.809469,2.043258,1.898874,1.910405,1.903951,1.651653,1.530006
10,1955,1.903475,1.933966,1.961608,1.832743,1.725020,2.010999,1.537940,1.545754,⋯,2.250129,1.578653,1.962877,1.845211,2.080186,1.937068,1.946695,1.937738,1.687500,1.565150


In [181]:
fwrite(edu4, "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/gpr_draws_country_year_collapsed_after_sort.csv")

In [98]:
# x<-data.table(test1)[location_id==6 & age_group_id==8 & sex_id==2, ]
# head(x)

In [99]:
ggplot(data.frame(x)) + geom_line(aes(x=as.numeric(year_id), y = draw0)) + 
    geom_line(aes(x=as.numeric(year_id), y = draw100))  + geom_line(aes(x=as.numeric(year_id), y = draw499))   + geom_line(aes(x=as.numeric(year_id), y = draw999)) 

ERROR: Error in parse(text = x, srcfile = src): <text>:2:0: unexpected end of input
1: ?
   ^
